In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


In [2]:
df = pd.read_csv('data/dataset.csv')
df = df.sort_values(by=['shop_id', 'item_id', 'date'])

In [4]:
columns = ['day','month','year','quarter','weekday','is_month_start','is_month_end','acc_month_num','category_id','price','quantity','pbi','OPEP_oil_price','unemployment_rate','IPC_rate']
grouped_data = df.groupby(['date', 'shop_id', 'item_id'])[columns].first().reset_index()
grouped_data.head()

,date,shop_id,item_id,day,month,year,quarter,weekday,is_month_start,is_month_end,acc_month_num,category_id,price,quantity,pbi,OPEP_oil_price,unemployment_rate,IPC_rate
0,2013-10-31,2,1495,31,10,2013,4,3,0,1,0,30,699.0,0,341693.0,106.75,5.5,6.3
1,2013-10-31,2,1555,31,10,2013,4,3,0,1,0,28,1299.0,0,341693.0,106.75,5.5,6.3
2,2013-10-31,2,1556,31,10,2013,4,3,0,1,0,20,2999.0,0,341693.0,106.75,5.5,6.3
3,2013-10-31,2,1855,31,10,2013,4,3,0,1,0,30,1199.0,0,341693.0,106.75,5.5,6.3
4,2013-10-31,2,1857,31,10,2013,4,3,0,1,0,20,2599.0,0,341693.0,106.75,5.5,6.3


In [3]:
unique_dates = df['date'].unique()
unique_shop_item = df[['shop_id', 'item_id']].drop_duplicates()
columns = ['day','month','year','quarter','weekday','is_month_start','is_month_end','acc_month_num','category_id','price','quantity','pbi','OPEP_oil_price','unemployment_rate','IPC_rate']
num_steps = len(unique_dates)
num_features = len(df.columns) - 3  # Excluyendo 'date', 'shop_id', 'item_id'

# Agrupa los datos por fecha, tienda y artículo
grouped_data = df.groupby(['date', 'shop_id', 'item_id'])[columns].first().reset_index()

# Crea un diccionario para mapear las fechas a índices en el tensor
date_index_mapping = {date: i for i, date in enumerate(unique_dates)}

# Inicializa un tensor con ceros
tensor = torch.zeros((len(unique_shop_item), num_steps, num_features))



# Llena el tensor con los valores correspondientes
for index, row in grouped_data.iterrows():
    date_index = date_index_mapping[row['date']]
    shop_item_index = unique_shop_item.index[(unique_shop_item['shop_id'] == row['shop_id']) & (unique_shop_item['item_id'] == row['item_id'])][0]
    numeric_data = row[columns].apply(pd.to_numeric, errors='coerce').values
    tensor[shop_item_index, date_index, :] = torch.tensor(numeric_data)

tensor.shape

torch.Size([6254, 731, 15])

In [4]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return self.data.shape[1] - self.sequence_length + 1

    def __getitem__(self, idx):
        end_idx = idx + self.sequence_length
        return {
            'data': self.data[:, idx:end_idx, :],
            'label': self.data[:, end_idx, 10]
        }

# Supongamos que tienes 'data' con forma (n_samples, n_timesteps, n_features)
# y 'labels' con las etiquetas correspondientes.
window_size = 30
x_train, x_test = train_test_split(tensor, test_size=0.2, random_state=42)
x_train, x_test = x_train[:-1], x_test[:-1]

# Creación de conjuntos de datos de series temporales
train_dataset = TimeSeriesDataset(x_train, sequence_length=window_size)
test_dataset = TimeSeriesDataset(x_test, sequence_length=window_size)

# Creación de DataLoader para facilitar el entrenamiento
train_dataloader = DataLoader(train_dataset, batch_size=window_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=window_size, shuffle=False)


In [5]:
# Prueba el DataLoader de entrenamiento
for batch in train_dataloader:
    print("Datos:", batch['data'].shape)
    break
    

Datos: torch.Size([30, 5002, 30, 15])


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        
        self.lstm = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # La entrada x debe tener dimensiones (batch_size, seq_len, input_size)
        lstm_out, _ = self.lstm(x)
        
        # Tomamos el último timestep de la salida de la LSTM
        last_timestep = lstm_out[:, -1, :]
        
        # Pasamos la salida del último timestep a través de una capa lineal
        output = self.fc(last_timestep)
        
        return output

# Definimos las dimensiones de entrada y salida de la red
input_size = 17  # Número de variables en cada timestep
hidden_size = 64  # Tamaño del estado oculto de la LSTM
num_layers = 2  # Número de capas en la LSTM
output_size = 1  # Tamaño de la salida (pronóstico para el próximo timestep)

# Creamos una instancia del modelo
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Definimos la pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


